# Import packages

In [8]:
#import matplotlib.pyplot as plt
#import numpy as np
#import sys
#import time
#import ROOT
#import pandas as pd
#from root_pandas import read_root
#from matplotlib.ticker import NullFormatter
#import matplotlib as mpl
#from iminuit import Minuit, describe, Struct
#import astroML.plotting as aml
from array import array
import itertools
import time

import matplotlib.pyplot as plt
import numpy as np

import astroML.plotting as aml
import iminuit
import pandas as pd
import root_pandas as rpd
from root_pandas import read_root
import ROOT
#from ROOT import TApplication as TAP
from matplotlib.ticker import NullFormatter


# Define Bins

In [9]:
zTbins = [0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2]
pTbins = [10, 11, 12.5, 16]

# Phi Projection & ZYAM

In [10]:
def GetPhiProj(filename, ipt, izt, Signal_DNN=True, doZYAM=True):
    DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN))
    file = ROOT.TFile(filename)
    histo2D = file.Get('DNN%i_Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
                       %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1],10*zTbins[izt],10*zTbins[izt+1]))

    Eta_Axis = histo2D.GetYaxis()
    PhiProjection = histo2D.ProjectionX('DNN%i_PhiProjection__pT_%1.0f_%1.0f__zt_%1.0f_%1.0f' 
                                  %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1],10*zTbins[izt],
                                    10*zTbins[izt+1]),Eta_Axis.FindBin(-0.6),Eta_Axis.FindBin(0.6));
    
    ntriggers = file.Get('N_DNN%i_Triggers_pT%1.0f_%1.0f' %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1]))
    PhiProjection.Scale(1.0/ntriggers.GetEntries())
    
    zyam = PhiProjection.Integral(11,13)/3
    print('ZYAM=%f'%(zyam))
    if(doZYAM):
        for i in range(25):
            y = PhiProjection.GetBinContent(i)
            y_error = PhiProjection.GetBinError(i)
            new_y = y-zyam
            PhiProjection.SetBinContent(i,new_y)
            #assumes 0 error in ZYAM
            new_y_error = y_error
            PhiProjection.SetBinError(i,new_y_error) 
    print(PhiProjection.GetEntries())        
    
#how do I plot ZYAM, in both cases?
# c.SaveAs("testPython.png")
    return PhiProjection

#### NOTE: The DNN corresponding to one photon is reflected in the naming scheme of histograms, where DNN1_Histo is signal (the same here) but background (DNN corresponding to 2 photons) is DNN2_Histo. Here, the DNN variable is a boolian corresponding to true for signal, false for not signal (a.k.a. background)

In [15]:
Sig=GetPhiProj('Same_Mix_Ratio.root',1,2,True,False)
print(Sig)

ZYAM=0.032232
3248.0
None


<img src="testPython.png">

# Low DNN Subtraction

In [5]:
def GetBKGsubtracted(filename, ptbin, ztbin, doZYAM=True):
    Signal = GetPhiProj(filename,ptbin,ztbin,True,doZYAM)
    Background = GetPhiProj(filename,ptbin,ztbin,False,doZYAM)
    #purity = GetBinContent(ptbin)
    purity = 0.4
    Background.Scale(purity)
    Signal.Add(Background,-1)
    c = TCanvas("canvas","canvas", 1000,800)
    Signal.Draw()
    c.SaveAs("Subtracted.pdf")
    return Signal
    

In [6]:
GetBKGsubtracted('Same_Mix_Ratio.root',1,3,True)

ZYAM=0.003671
463.0
ZYAM=0.003033
434.0


AttributeError: 'PyROOT_NoneType' object has no attribute 'Scale'